## ROSLab: <u>RO</u>botics <u>S</u>oftware with Jupyter<u>Lab</u>
ROSLab is a [JupyterLab](https://jupyterlab.readthedocs.io/en/stable/)
environment for a source code repository with robotics software. 
It automatically creates a [Dockerfile](https://docs.docker.com/engine/reference/builder/) from a simple UI form.

Please select the cell below and press simultaneously `Ctrl+Enter`. Then fill in the form fields, and press the `Proceed` button.

In [ ]:
import form
form.ui